In [1]:
from __future__ import print_function
import streamlit as st
import collections
import os
import io
import math
import operator
import sys
import networkx as nx
from preprocess import cleanText

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hardik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
window = 10
numberofSentences = 6
nodeHash = {}
textRank = {}
sentenceDictionary = collections.defaultdict(dict)
size = 0
sentences = []


In [3]:
def generatepositionaldistribution():
    global nodeHash, sentenceDictionary, sentences, size
    positional_dictionary = collections.defaultdict(dict)
    count = 0
    for i in sentenceDictionary.keys():
        for j in range(0, len(sentenceDictionary[i])):
            count += 1
            position = float(count) / (float(size) + 1.0)
            positional_dictionary[i][j] = 1.0 / \
                (math.pi * math.sqrt(position * (1 - position)))
            word = sentenceDictionary[i][j]
            if word in nodeHash:
                if nodeHash[word] < positional_dictionary[i][j]:
                    nodeHash[word] = positional_dictionary[i][j]
            else:
                nodeHash[word] = positional_dictionary[i][j]

In [4]:
def textrank():
    '''
        Generates a graph based ranking model for the tokens
    :return: Keyphrases that are most relevant for generating the summary.
    '''
    global sentenceDictionary, nodeHash, textRank
    graph = nx.Graph()
    graph.add_nodes_from(nodeHash.keys())
    for i in sentenceDictionary.keys():
        for j in range(0, len(sentenceDictionary[i])):
            current_word = sentenceDictionary[i][j]
            next_words = sentenceDictionary[i][j + 1:j + window]
            for word in next_words:
                graph.add_edge(current_word, word, weight=(
                    nodeHash[current_word] + nodeHash[word]) / 2)
    textRank = nx.pagerank(graph, weight='weight')
    keyphrases = sorted(textRank, key=textRank.get, reverse=True)[:n]
    return keyphrases

In [62]:
def summarize(filepath, keyphrases, numberofSentences):
    '''
        Generates the summary and writes the summary to the file.
    :param filePath: path of file to be used for summarization.
    :param keyphrases: Extracted keyphrases
    :param numberofSentences: Number of sentences needed as a summary
    :output: Writes the summary to the file
    '''
    global textRank, sentenceDictionary, sentences
    sentenceScore = {}
    for i in sentenceDictionary.keys():
        position = float(i + 1) / (float(len(sentences)) + 1.0)
        if position <= 0.0 or position >= 1.0:
            continue
        try:
            positionalFeatureWeight = 1.0 / \
                (math.pi * math.sqrt(position * (1.0 - position)))
        except ValueError:
            continue
        sumKeyPhrases = 0.0
        for keyphrase in keyphrases:
            if keyphrase in sentenceDictionary[i]:
                sumKeyPhrases += textRank[keyphrase]
        sentenceScore[i] = sumKeyPhrases * positionalFeatureWeight
    sortedSentenceScores = sorted(sentenceScore.items(
    ), key=operator.itemgetter(1), reverse=True)[:numberofSentences]
    sortedSentenceScores = sorted(
        sortedSentenceScores, key=operator.itemgetter(0), reverse=False)
    print("\nSummary: ")
    summary = []
    arr = []
    # for keyphrase in keyphrases:
    #     print(keyphrase)
    # print(keyphrases)

    for i in range(0, len(sortedSentenceScores)):
        arr.append(sentences[sortedSentenceScores[i][0]])
    s = "".join(arr)
    # print(s)
    return (s)

In [63]:
text = "D:\Codes\Final Project\sample1.txt"

In [64]:
def process(arg1):
    '''
    :param arg1: path to the file containing the text to be summarized
    :param arg2: Number of sentences to be extracted as summary
    :param arg3: size of the window to be used in the co-occurance
    '''
    arg2 = 5
    arg3 = 6
    global window, n, numberofSentences, textRank, sentenceDictionary, size, sentences
    if arg1 != None and arg2 != None and arg3 != None:
        sentenceDictionary, sentences, size = cleanText(arg1)
        window = int(arg3)
        numberofSentences = int(arg2)
        n = int(math.ceil(min(0.1 * size, 7 * math.log(size))))
        generatepositionaldistribution()
        keyphrases = textrank()
        t = summarize(arg1, keyphrases, numberofSentences)
        return (t)
    else:
        print("not enough parameters")

In [65]:
process(text)


Summary: 


'ही तर साली आपल्या रस्त्याला राहणारी.रंडी साली वीकेंडला गांड मटकवत आणि बॉल मिरवत हिंडायची मॉलमध्ये.हिकडे साली बायको आपल्याला कंटाळून माहेरी गेली.जाताना खोदलेल्या खड्डयात घसरली नं साली.उचलायच्या, हात द्याच्या नावानं हितंतितं हात लावून घेतला सालीला!'

In [52]:
ह्या पोरी साल्या अशाच. ती आधीच्या कंपनीतली पोरगी गुमान झोपली, पण मग सालीनं हरासमेंटची कंप्लेंट केली. आगोदरच पेपर टाकले होते म्हणून वाचलो. 
मग साली लांब नोकरी धरली, वीस किलोमीटरवर. इथे लागलो तर टेसात टेस्ट लीड म्हणून. 
साल्या नव्या कोऱ्या पोरी आपल्याखाली - हा हा हा - हाताखाली काम करायला. मुद्दामून गावाकडून आलेल्या पोरी घ्यायच्या. 
रात्रीच्या शिफ्टला घ्यायच्या. नाय तर रात्री जास्त काम करायला म्हणून बसवून घ्यायच्या. आपली स्टायल तर अशी की, पोरी तं पार खल्लास होणार.



SyntaxError: invalid syntax (1737209423.py, line 1)